In [6]:
#Import libraries and set random seed
import pandas as pd
import numpy as np
#import multiprocessing

#multiprocessing.cpu_count()
#import torch
import os
import pandas as pd
#from torchvision.io import read_image
import numpy as np
import matplotlib.pyplot as plt
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils.data import Dataset, DataLoader, random_split
#from torchvision import transforms
#from torch import nn, optim
#import torch.nn.functional as F
#import torch.utils.data as data
#import torch
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


np.random.seed(42)

In [7]:
#Read In Olink Data, Format, and Find Percent Missing
ukolink = pd.read_csv("/home/ffc4001/ukbiobankdata/olink_data.txt",sep='\t')
ukolink1 = ukolink[ukolink["ins_index"] == 0]
ukolink1.drop(["ins_index"], axis=1, inplace=True)
ukolink1 = ukolink1.pivot_table(values='result', columns='protein_id', index = "eid", aggfunc='first')
percent_missing = ukolink1.isnull().sum() * 100 / len(ukolink1)
missing_value_df = pd.DataFrame({'column_name': ukolink1.columns,
                                 'percent_missing': percent_missing})

/tmp/ipykernel_793386/959808266.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukolink1.drop(["ins_index"], axis=1, inplace=True)


In [8]:
#Show Percent Missing
missing_value_df.sort_values('percent_missing')

,column_name,percent_missing
protein_id,,
619,619,1.017001
1881,1881,1.017001
361,361,1.017001
2397,2397,1.017001
2381,2381,1.017001
...,...,...
115,115,8.860808
2614,2614,13.378491
732,732,23.170917


In [10]:
#Replace uniprot_id's with protein names
olinknames = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/coding143.tsv",sep='\t')
olinknames_dict = dict(zip(olinknames.coding, olinknames.meaning))

In [103]:
olinknames_dict[2297]

'RNASET2;Ribonuclease T2'

In [11]:
ukolink1.columns = ukolink1.columns.map(olinknames_dict)

In [12]:
ukolink1

protein_id,AARSD1;Alanyl-tRNA editing protein Aarsd1,ABHD14B;Protein ABHD14B,ABL1;Tyrosine-protein kinase ABL1,"ACAA1;3-ketoacyl-CoA thiolase, peroxisomal",ACAN;Aggrecan core protein,ACE2;Angiotensin-converting enzyme 2,ACOX1;Peroxisomal acyl-coenzyme A oxidase 1,ACP5;Tartrate-resistant acid phosphatase type 5,ACP6;Lysophosphatidic acid phosphatase type 6,"ACTA2;Actin, aortic smooth muscle",...,WNT9A;Protein Wnt-9a,WWP2;NEDD4-like E3 ubiquitin-protein ligase WWP2,XCL1;Lymphotactin,XG;Glycoprotein Xg,XPNPEP2;Xaa-Pro aminopeptidase 2,XRCC4;DNA repair protein XRCC4,YES1;Tyrosine-protein kinase Yes,YTHDF3;YTH domain-containing family protein 3,ZBTB16;Zinc finger and BTB domain-containing protein 16,ZBTB17;Zinc finger and BTB domain-containing protein 17
eid,,,,,,,,,,,,,,,,,,,,,
1000048,-0.44185,NaN,-0.37680,1.83095,NaN,NaN,NaN,NaN,-0.49450,NaN,...,0.9734,NaN,1.35320,NaN,-1.32800,NaN,-0.7925,-1.18150,-0.58535,NaN
1000072,0.26705,0.19705,1.36915,0.89980,0.45650,1.75230,0.31135,0.24905,2.10985,0.68200,...,0.2045,0.9581,-0.58690,-0.25035,0.35035,-0.34695,2.3289,1.49335,1.39360,-0.13565
1000256,-0.08155,0.23025,0.16140,-0.65685,0.06830,-0.08315,0.16710,-0.43970,0.54610,0.42175,...,0.4965,-0.2325,-0.50680,0.23510,0.03730,-0.06620,0.5781,0.60500,-0.43985,-0.13210
1000305,-0.29995,0.34875,0.53075,2.46510,-0.17530,0.89800,1.50775,-0.02125,-0.58265,0.63660,...,-0.2612,0.5441,-0.77760,0.15825,-0.94215,0.37285,0.9376,0.11665,0.65120,-0.09725
1000364,0.15620,-0.72080,-0.48390,-1.04680,-0.16280,-0.09300,0.01440,-0.25280,0.10330,-0.02770,...,0.1755,-0.6188,-0.59860,-0.42140,0.76950,-0.42335,-0.9590,-1.47930,-0.26205,-0.25430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6024705,-0.73905,0.36930,-0.14390,1.50760,0.21720,0.28195,-0.43570,-0.10540,-0.14640,0.15380,...,NaN,-0.6455,-0.31160,0.54810,1.25920,-0.07475,-1.5997,NaN,-0.83605,0.28210
6024773,-0.11700,-0.02110,-1.09680,1.79300,0.25010,0.32895,0.29840,0.07500,0.09850,0.53850,...,0.1482,-1.0865,-0.42150,-0.35830,-2.50120,0.31715,-3.6805,-0.49145,-0.78885,0.31050
6025049,-0.61460,0.60790,0.11420,-0.52910,0.51750,-0.28175,-0.27260,0.10640,-0.63840,0.20410,...,0.5065,0.0827,0.03530,0.71250,-0.14010,1.08485,0.0000,0.98685,-0.51745,1.13160


In [24]:
eid = ukolink1.index
#eid.to_csv("/home/ffc4001/ukbiobankdata/Data/olink_eid.csv")

Int64Index([1000048, 1000072, 1000256, 1000305, 1000364, 1000543, 1000687,
            1000700, 1000793, 1000834,
            ...
            6024050, 6024247, 6024295, 6024456, 6024670, 6024705, 6024773,
            6025049, 6025243, 6025336],
           dtype='int64', name='eid', length=52704)

In [ ]:
# Create correlation matrix WORK IN PROGRESS
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df.drop(to_drop, axis=1, inplace=True)

In [28]:
#Read in Variant only data
Trn_input =  pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ukbiobank_variantsonlytraindata_eid.csv")
Trn_target = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ukbiobank_variantsonlytrainy_eid.csv")
Test_input = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ukbiobank_variantsonlytestdata_eid.csv")
Test_target = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ukbiobank_variantsonlytesty_eid.csv")
Val_input = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ppmi_variantsonlyvaldata_eid.csv")
Val_target = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ppmi_variantsonlyvaly_eid.csv")
Trn_input = pd.concat([Trn_input, pd.DataFrame(Trn_target)], axis=1)
Test_input = pd.concat([Test_input, pd.DataFrame(Test_target)], axis=1)


In [587]:
Val_input

,Age,Inferred_Gender,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,X1.155044197_T_G_G,...,X9.33778399_G_A_G,X9.34046391_C_T_C,X9.5023192_A_G_G,X9.5065003_C_G_G,X9.86258685_A_C_C,X9.86945961_A_C_C,X9.89056018_A_G_G,X9.89059858_G_A_A,X9.93256092_T_C_T,eid_correct
0,69.139726,0.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,2,2,1,3000.0
1,65.142466,1.0,1,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3001.0
2,67.578082,0.0,2,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,3002.0
3,56.717808,0.0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3003.0
4,59.413699,1.0,1,1,0,0,1,0,1,0,...,1,0,0,0,1,0,1,1,0,3004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,71.117808,1.0,1,0,0,1,2,0,0,0,...,1,0,0,0,0,0,2,2,1,4124.0
425,64.019178,1.0,1,0,0,1,0,1,1,0,...,0,0,0,0,1,0,0,1,0,4125.0
426,55.734247,1.0,1,0,0,0,1,1,1,0,...,1,0,0,0,0,0,1,1,1,4126.0
427,55.764384,1.0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,1,1,2,4136.0


In [29]:
Trn_input.drop(["Unnamed: 0"], axis=1, inplace=True) 
Trn_target.drop(["Unnamed: 0"], axis=1, inplace=True) 
Test_input.drop(["Unnamed: 0"], axis=1, inplace=True)
Test_target.drop(["Unnamed: 0"], axis=1, inplace=True) 
Val_input.drop(["Unnamed: 0"], axis=1, inplace=True)
Val_target.drop(["Unnamed: 0"], axis=1, inplace=True) 

In [30]:
ukolink1.index.names = ['eid_correct']
ukblood_values = ukolink1

In [31]:
#Merge Olink Data to PPMI Data
Trn_input = Trn_input.merge(ukblood_values,how='inner', on="eid_correct")
Test_input = Test_input.merge(ukblood_values,how='inner', on="eid_correct")

In [32]:
Trn_target = Trn_input["Y"]
Test_target = Test_input["Y"]

In [33]:
Trn_input.drop(["Y"], axis=1, inplace=True)
Test_input.drop(["Y"], axis=1, inplace=True)

In [73]:
Trn_input.iloc[:,800:850]

,CASP10;Caspase-10,CASP2;Caspase-2,CASP3;Caspase-3,CASP8;Caspase-8,CBLIF;Cobalamin binding intrinsic factor,CBLN4;Cerebellin-4,CC2D1A;Coiled-coil and C2 domain-containing protein 1A,CCDC80;Coiled-coil domain-containing protein 80,CCL11;Eotaxin,CCL13;C-C motif chemokine 13,...,CD160;CD160 antigen,CD163;Scavenger receptor cysteine-rich type 1 protein M130,CD164;Sialomucin core protein 24,CD177;CD177 antigen,CD1C;T-cell surface glycoprotein CD1c,CD200;OX-2 membrane glycoprotein,CD200R1;Cell surface glycoprotein CD200 receptor 1,CD207;C-type lectin domain family 4 member K,CD209;CD209 antigen,CD22;B-cell receptor CD22
0,-0.28695,0.0000,-0.0016,0.00140,1.07740,0.00000,-0.13495,0.41775,0.65750,1.00955,...,-0.10440,-0.0174,0.24515,-0.06610,0.3807,0.40565,0.3158,1.06040,0.00080,0.16810
1,1.57665,1.5356,1.5627,1.78680,0.06980,0.31760,1.80295,1.67305,0.28510,-0.23220,...,2.65750,0.5856,-0.00525,-0.00510,0.0115,1.33315,1.1396,0.78610,0.64930,0.33950
2,0.76475,-0.2135,0.0554,0.58195,-0.55820,0.06260,1.03485,-0.29515,0.32275,0.44095,...,-0.27230,-0.3406,-0.16205,0.11110,0.2081,0.07195,-0.0221,0.00000,-0.27520,-0.17520
3,0.23775,-0.2892,-0.5930,-1.06400,1.19350,0.24960,-0.05695,-0.00235,0.37020,0.14375,...,-0.23680,-0.5124,-0.27865,-1.07460,0.3056,-0.21905,-0.4988,0.49260,0.42430,-0.54560
4,-1.27310,-1.8406,-2.2022,-1.10340,0.36285,0.03850,-0.70695,0.41095,-0.03765,-0.30330,...,-0.39765,0.2467,0.07515,-1.35355,-0.3070,-0.19440,-0.0057,0.01895,0.81515,-0.30215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26068,1.86200,1.2200,0.7017,1.13160,-0.83260,-1.34825,2.32645,-0.45145,-0.42440,-0.59110,...,-0.05970,0.2023,-0.06255,-0.03150,0.0082,0.05185,0.0327,0.11680,0.45100,0.22795
26069,-0.19320,-0.1413,0.4992,-0.07070,0.38820,0.22195,0.43705,0.25965,-0.26990,0.95410,...,0.18590,-0.6445,-0.35845,0.66270,-0.0226,0.04445,-0.4702,-0.15810,-0.16650,0.41765
26070,-0.39115,0.6251,0.6614,0.09090,0.41360,0.18710,0.51785,0.08245,0.04390,-0.25920,...,0.12400,0.5305,-0.16365,0.55810,-0.1715,-0.25955,-0.2016,-0.54350,-0.42320,0.79260
26071,0.21975,0.3500,0.2850,0.28940,0.19500,0.23000,0.43615,0.15715,0.01980,-1.03935,...,0.42280,0.0837,0.08155,0.50190,-0.4377,-0.16595,0.4512,-0.12920,-0.40880,0.32730


In [36]:
Test_input

,eid_correct,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,...,WNT9A;Protein Wnt-9a,WWP2;NEDD4-like E3 ubiquitin-protein ligase WWP2,XCL1;Lymphotactin,XG;Glycoprotein Xg,XPNPEP2;Xaa-Pro aminopeptidase 2,XRCC4;DNA repair protein XRCC4,YES1;Tyrosine-protein kinase Yes,YTHDF3;YTH domain-containing family protein 3,ZBTB16;Zinc finger and BTB domain-containing protein 16,ZBTB17;Zinc finger and BTB domain-containing protein 17
0,3044277.0,1,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0949,-0.8319,-0.51690,-0.57360,0.26810,-0.74945,-0.6714,-1.66505,0.27345,0.18650
1,2092473.0,1,53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0302,-0.9410,0.16895,-0.32510,-0.08750,-0.22335,-1.7270,-0.86890,-0.86935,-0.58020
2,2201664.0,1,69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4265,0.2265,0.27950,-0.27475,0.07585,-0.37935,0.6809,0.46975,0.65270,0.17745
3,5075208.0,1,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0190,-0.3626,0.00635,0.01950,-0.59780,-0.11445,-0.6602,1.07940,-0.19845,0.15830
4,1262752.0,1,66,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.1431,-0.1234,0.36590,-0.59260,0.14490,-0.44585,-0.7857,0.11930,-0.19275,-0.19380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,4827711.0,1,64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1348,0.4933,-0.72360,0.10840,-1.26810,NaN,0.7743,2.24960,-0.01835,-0.25230
17331,3735210.0,0,69,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.4130,-0.2031,-0.60625,0.25610,-0.14920,-0.14885,-0.0388,0.35900,-0.50545,0.24840
17332,5941955.0,0,44,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0531,-0.3950,-0.61595,0.10330,0.36000,0.01115,-0.4016,-1.01480,0.01395,-0.31040
17333,5127128.0,0,67,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2045,0.1034,0.26015,0.45970,0.91660,-0.07995,0.2354,0.13150,0.26515,-0.29610


In [42]:
#eid_correct1 = Trn_input.eid_correct
#eid_correct2 = Test_input.eid_correct

In [48]:
Trn_input.drop(["eid_correct"], axis=1, inplace=True)
Test_input.drop(["eid_correct"], axis=1, inplace=True)

In [49]:
#Imputation
Trn_input.fillna(Trn_input.mode().iloc[0], inplace=True)
Test_input.fillna(Test_input.mode().iloc[0], inplace=True)

In [ ]:
#age plus olink normalization
column = "age"

In [ ]:
# #Standarize age, and biomarkers
# Trn_input[column] = (Trn_input[column] - Trn_input[column].min()) / (Trn_input[column].max() - Trn_input[column].min())
# Test_input[column] = (Test_input[column] - Test_input[column].min()) / (Test_input[column].max() - Test_input[column].min())
# X_val[column] = (X_val[column] - X_val[column].min()) / (X_val[column].max() - X_val[column].min())

In [43]:
#eid_correct1

0        5272331.0
1        2685674.0
2        2212508.0
3        4933558.0
4        3164758.0
           ...    
26068    4683650.0
26069    5511678.0
26070    4643173.0
26071    1321441.0
26072    1263493.0
Name: eid_correct, Length: 26073, dtype: float64

In [45]:
#eid3 = pd.concat([eid_correct1, eid_correct2])

In [46]:
#eid3

0        5272331.0
1        2685674.0
2        2212508.0
3        4933558.0
4        3164758.0
           ...    
17330    4827711.0
17331    3735210.0
17332    5941955.0
17333    5127128.0
17334    5126869.0
Name: eid_correct, Length: 43408, dtype: float64

In [47]:
#eid3.to_csv("/home/ffc4001/ukbiobankdata/Data/olink_eid.csv")

In [ ]:
#SAVE FILES

In [ ]:
#add olink to ppmi data set align.

In [50]:
X_val = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_valvariantsonly")


In [532]:
Y_val = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_valvariantsonly")

In [58]:
ppmi_id = Val_input[["eid_correct"]]

In [59]:
ppmi_id

,eid_correct
0,3000.0
1,3001.0
2,3002.0
3,3003.0
4,3004.0
...,...
424,4124.0
425,4125.0
426,4126.0
427,4136.0


In [60]:
X_val["PATNO"] = ppmi_id["eid_correct"]

In [61]:
X_val

,Age,Inferred_Gender,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,X1.155044197_T_G_G,...,X9.33778399_G_A_G,X9.34046391_C_T_C,X9.5023192_A_G_G,X9.5065003_C_G_G,X9.86258685_A_C_C,X9.86945961_A_C_C,X9.89056018_A_G_G,X9.89059858_G_A_A,X9.93256092_T_C_T,PATNO
0,0.735751,0.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,2,2,1,3000.0
1,0.659409,1.0,1,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3001.0
2,0.705926,0.0,2,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,3002.0
3,0.498510,0.0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3003.0
4,0.549998,1.0,1,1,0,0,1,0,1,0,...,1,0,0,0,1,0,1,1,0,3004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,0.773529,1.0,1,0,0,1,2,0,0,0,...,1,0,0,0,0,0,2,2,1,4124.0
425,0.637956,1.0,1,0,0,1,0,1,1,0,...,0,0,0,0,1,0,0,1,0,4125.0
426,0.479726,1.0,1,0,0,0,1,1,1,0,...,1,0,0,0,0,0,1,1,1,4126.0
427,0.480301,1.0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,1,1,2,4136.0


In [472]:
olink_cardio_ppmi = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/PPMI_Project_196_Plasma_Cardio_NPX_13Dec2023.csv")
olink_immune_ppmi = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/PPMI_Project_196_Plasma_INF_NPX_13Dec2023.csv")
olink_cancer_ppmi = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/PPMI_Project_196_Plasma_ONC_NPX_13Dec2023.csv")
olink_neuro_ppmi = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/PPMI_Project_196_Plasma_NEURO_NPX_13Dec2023.csv")

In [480]:
olink_immune_ppmi = olink_immune_ppmi.loc[olink_immune_ppmi['EVENT_ID'] == "BL"]
olink_cardio_ppmi = olink_cardio_ppmi.loc[olink_cardio_ppmi['EVENT_ID'] == "BL"]
olink_cancer_ppmi = olink_cancer_ppmi.loc[olink_cancer_ppmi['EVENT_ID'] == "BL"]
olink_neuro_ppmi = olink_neuro_ppmi.loc[olink_neuro_ppmi['EVENT_ID'] == "BL"]

In [476]:
len(bl["PATNO"].unique())

227

In [349]:
#olink_cardio_ppmi.EVENT_ID.value_counts()

BL     97047
V06    60147
V10    54612
V04    46863
V02    40590
V08    34317
V12    20295
V01     9594
V07     9225
V05     8856
ST      8118
V03     7011
V09     6642
V11     5166
V13     2583
V14     1845
V15      369
Name: EVENT_ID, dtype: int64

PPMI-85242    5166
PPMI-90456    4797
PPMI-60013    4797
PPMI-60107    4797
PPMI-60065    4797
              ... 
PPMI-3636     1107
PPMI-3424     1107
PPMI-3807     1107
PPMI-3428     1107
PPMI-3362     1107
Name: PATNO, Length: 99, dtype: int64

In [437]:
#temp2 = olink_cardio_ppmi[olink_cardio_ppmi["PATNO"] == "PPMI-3907"]

AttributeError: 'DataFrame' object has no attribute 'unique'

In [351]:
#temp1

,PATNO,EVENT_ID,INDEX,OLINKID,UNIPROT,ASSAY,MISSINGFREQ,PANEL,PANEL_LOT_NR,PLATEID,QC_WARNING,LOD,NPX,update_stamp
7380,PPMI-3374,ST,79,OID20321,O00584,RNASET2,0.0,Cardiometabolic,B04409,20201991_SS200108_CARDIO_INF,PASS,-6.5023,0.7436,2021-04-12 10:50:32.0
7381,PPMI-3374,ST,79,OID20206,Q07108,CD69,0.0,Cardiometabolic,B04409,20201991_SS200108_CARDIO_INF,PASS,-4.2032,2.1978,2021-04-12 10:50:32.0
7382,PPMI-3374,ST,79,OID20195,P35754,GLRX,0.0,Cardiometabolic,B04409,20201991_SS200108_CARDIO_INF,PASS,-2.6444,1.0071,2021-04-12 10:50:32.0
7383,PPMI-3374,ST,79,OID20094,Q9H5Y7,SLITRK6,0.0,Cardiometabolic,B04409,20201991_SS200108_CARDIO_INF,PASS,-1.6741,1.1089,2021-04-12 10:50:32.0
7384,PPMI-3374,ST,79,OID20216,P31431,SDC4,0.0,Cardiometabolic,B04409,20201991_SS200108_CARDIO_INF,PASS,-5.4201,-0.8836,2021-04-12 10:50:32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383017,PPMI-3793,ST,59,OID20398,P17936,IGFBP3,0.0,Cardiometabolic,B14808,BIOREP_pl2_Samplesheet,PASS,-7.2072,0.4301,2022-12-12 12:37:07.0
383018,PPMI-3793,ST,59,OID20399,P36222,CHI3L1,0.0,Cardiometabolic,B14808,BIOREP_pl2_Samplesheet,PASS,-4.1130,-0.0938,2022-12-12 12:37:07.0
383019,PPMI-3793,ST,59,OID20396,P19320,VCAM1,0.0,Cardiometabolic,B14808,BIOREP_pl2_Samplesheet,PASS,-4.2048,0.0073,2022-12-12 12:37:07.0
383020,PPMI-3793,ST,59,OID20397,Q9NPY3,CD93,0.0,Cardiometabolic,B14808,BIOREP_pl2_Samplesheet,PASS,-7.1830,0.3515,2022-12-12 12:37:07.0


In [438]:
#temp2[temp2["ASSAY"] == "CNPY2"]

,PATNO,EVENT_ID,INDEX,OLINKID,UNIPROT,ASSAY,MISSINGFREQ,PANEL,PANEL_LOT_NR,PLATEID,QC_WARNING,LOD,NPX,update_stamp
125474,PPMI-3907,V08,26,OID20069,Q9Y2B0,CNPY2,0.0902,Cardiometabolic,B14808,BIOREP_pl2_Samplesheet,PASS,-0.5238,1.1837,2022-12-12 12:37:07.0
377492,PPMI-3907,V01,43,OID20069,Q9Y2B0,CNPY2,0.0902,Cardiometabolic,B14808,BIOREP_pl2_Samplesheet,PASS,-0.5238,1.1718,2022-12-12 12:37:07.0
383396,PPMI-3907,V04,17,OID20069,Q9Y2B0,CNPY2,0.0902,Cardiometabolic,B14808,BIOREP_pl2_Samplesheet,PASS,-0.5238,0.7350,2022-12-12 12:37:07.0


In [ ]:
#Match to ukbiobank column name
#I need patno by NPX with column names ASSAY

In [125]:
#olink_neuro_ppmi['PATNO']. nunique()

295

In [481]:
olink_cardio_ppmi1 = olink_cardio_ppmi[["PATNO", "ASSAY", "NPX"]]
olink_immune_ppmi1 = olink_immune_ppmi[["PATNO", "ASSAY", "NPX"]]
olink_neuro_ppmi1 = olink_neuro_ppmi[["PATNO", "ASSAY", "NPX"]]
olink_cancer_ppmi1 = olink_cancer_ppmi[["PATNO", "ASSAY", "NPX"]]

In [482]:
olink_cardio_ppmi1

,PATNO,ASSAY,NPX
738,PPMI-3325,RNASET2,-0.2406
739,PPMI-3325,CD69,-1.1331
740,PPMI-3325,GLRX,0.3183
741,PPMI-3325,SLITRK6,0.4826
742,PPMI-3325,SDC4,-4.0025
...,...,...,...
425821,PPMI-60056,NCAM1,0.5491
425822,PPMI-60056,LBP,-0.3056
425823,PPMI-60056,ITIH3,0.3805
425824,PPMI-60056,HYAL1,-0.0024


In [ ]:
#Take the BL for each patno, if no BL take V02 if no V02 take 

In [ ]:
#Drop ST
#RECODE 0:15 bL to V10


In [410]:
#event_id_dict = {"BL":0, "V01":1,"V02":2,"V03":3,"V04":4,"V05":5,"V06":6,"V07":7,"V08":8,"V09":9,"V10":10,"V11":11,"V12":12,"V13":13,"V14":14,"V15":15, "ST":16}



In [411]:
#olink_cardio_ppmi1=olink_cardio_ppmi1.replace({"EVENT_ID": event_id_dict})

In [415]:
#olink_cardio_ppmi1

,PATNO,EVENT_ID,ASSAY,NPX
0,PPMI-3374,10.0,RNASET2,0.8018
1,PPMI-3374,10.0,CD69,3.0466
2,PPMI-3374,10.0,GLRX,1.3235
3,PPMI-3374,10.0,SLITRK6,1.3717
4,PPMI-3374,10.0,SDC4,-0.5932
...,...,...,...,...
428035,PPMI-3453,10.0,PLA2G1B,0.5628
428036,PPMI-3453,10.0,CD55,0.6982
428037,PPMI-3453,10.0,IGFBP6,0.2229
428038,PPMI-3453,10.0,ITGB1,0.7205


In [416]:
#olink_cardio_ppmi1["EVENT_ID"] = pd.to_numeric(olink_cardio_ppmi1["EVENT_ID"])

In [421]:
#olink_cardio_ppmi1["EVENT_ID"].fillna(17, inplace=True)

In [422]:
#olink_cardio_ppmi1["EVENT_ID"].isna().sum()

0

In [423]:
t#emp1 = olink_cardio_ppmi1.join(olink_cardio_ppmi1.groupby("PATNO")['EVENT_ID'].agg('min').on='PATNO')


SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (1024150814.py, line 1)

In [435]:
#temp1 = olink_cardio_ppmi1.loc[olink_cardio_ppmi1.groupby('PATNO')['EVENT_ID'].idxmin()]

In [451]:
#temp1.sort_values(by=['EVENT_ID'])[260:]

,PATNO,EVENT_ID,ASSAY,NPX
318816,PPMI-3057,2.0,HK2,0.6591
300735,PPMI-4104,2.0,CNPY2,0.3257
354609,PPMI-3806,4.0,TCL1B,0.3377
387450,PPMI-3132,4.0,AKR1C4,-1.1788
143172,PPMI-60091,4.0,VAMP5,0.9103
361989,PPMI-3756,4.0,CTF1,2.5976
350919,PPMI-3369,4.0,TCL1B,-0.7658
301473,PPMI-3216,4.0,MTPN,0.2402
104058,PPMI-60057,4.0,CTF1,0.3846
199629,PPMI-3320,4.0,TNNI3,2.8271


In [483]:
olink_cardio_ppmi1

,PATNO,ASSAY,NPX
738,PPMI-3325,RNASET2,-0.2406
739,PPMI-3325,CD69,-1.1331
740,PPMI-3325,GLRX,0.3183
741,PPMI-3325,SLITRK6,0.4826
742,PPMI-3325,SDC4,-4.0025
...,...,...,...
425821,PPMI-60056,NCAM1,0.5491
425822,PPMI-60056,LBP,-0.3056
425823,PPMI-60056,ITIH3,0.3805
425824,PPMI-60056,HYAL1,-0.0024


In [484]:
olink_cardio_ppmi1.iloc[-100:,0:3]

,PATNO,ASSAY,NPX
425726,PPMI-60056,CD59,0.6151
425727,PPMI-60056,AMY2B,-0.0028
425728,PPMI-60056,TCN2,0.3546
425729,PPMI-60056,SERPINE1,0.8101
425730,PPMI-60056,MCAM,0.8061
...,...,...,...
425821,PPMI-60056,NCAM1,0.5491
425822,PPMI-60056,LBP,-0.3056
425823,PPMI-60056,ITIH3,0.3805
425824,PPMI-60056,HYAL1,-0.0024


In [485]:
olink_cardio_ppmi2 = olink_cardio_ppmi1.pivot_table(['NPX'], 'PATNO','ASSAY')
olink_neuro_ppmi2 = olink_neuro_ppmi1.pivot_table(['NPX'], 'PATNO','ASSAY')
olink_cancer_ppmi2 = olink_cancer_ppmi1.pivot_table(['NPX'], 'PATNO','ASSAY')
olink_immune_ppmi2 = olink_immune_ppmi1.pivot_table(['NPX'], 'PATNO','ASSAY')

In [486]:
olink_cancer_ppmi2

NPX                                                             \
ASSAY       AARSD1    ABL1   ACAA1    ACP6 ADAMTS15 ADAMTS8 ADCYAP1R1  ADGRG1   
PATNO                                                                           
PPMI-12224  0.3978  0.3545 -1.3954  1.0728   3.3188  1.0104   -0.3453  1.0168   
PPMI-12499  2.9192  2.1646  1.2238  1.2234   2.9050  0.3143    0.4974  4.1220   
PPMI-12593  3.7517  3.8234  2.5279  1.1390   3.8457  1.7085   -0.1460  0.8952   
PPMI-13039  1.9736  1.7247  0.5997  0.8770   3.3143  0.8214    0.4348  2.1407   
PPMI-13424  0.7901  0.8931 -0.2197  0.9044   3.4085  1.2802   -2.0693 -0.3079   
...            ...     ...     ...     ...      ...     ...       ...     ...   
PPMI-90456  2.0204  2.6473  0.3890  1.3523   3.1578  1.3138   -0.0015  1.8442   
PPMI-91097  2.9806  1.9931  1.9430  0.8259   3.0793 -0.1127   -1.6191  1.4559   
PPMI-91837  3.6136  3.4301  2.3686  0.5248   3.8056  0.2230   -0.2413  1.8540   
PPMI-92490  3.2302  1.7680  0.5892  0.7021   3.2030  0.8842    0.1209  3.6206   
PPMI-92834  4.2614  3.3059  0.7017  0.9950   3.1914  0.4206   -0.1275  1.7286   

                            ...                                                \
ASSAY          ADM    AGR3  ...   VTCN1    VWA1  WFDC12   WFDC2    WIF1 WISP1   
PATNO                       ...                                                 
PPMI-12224  1.5285  0.9903  ...  0.2638  0.8958  0.0673  1.0621  0.6804   NaN   
PPMI-12499  1.5271 -3.8330  ... -1.1456 -0.1553  0.0247  0.7898  0.4746   NaN   
PPMI-12593  1.7712 -0.1343  ... -0.3820  0.6267  0.4018  1.3762  0.4968   NaN   
PPMI-13039  1.6113  0.3798  ...  0.4420  0.5331  0.8172  1.4060  0.5617   NaN   
PPMI-13424  1.4587 -2.0356  ...  0.2966 -0.3746  0.6601  0.7001  1.6078   NaN   
...            ...     ...  ...     ...     ...     ...     ...     ...   ...   
PPMI-90456  1.8755  0.7587  ...  0.9104  0.1100  0.9709  2.2826  0.7434   NaN   
PPMI-91097  1.4954 -0.5183  ... -0.2225  0.8913 -0.2045  0.8504 -0.5513   NaN   
PPMI-91837  1.7689 -0.1283  ...  0.2376  0.5181  0.4468  2.0338  0.4524   NaN   
PPMI-92490  1.9094  0.2387  ...  1.1262 -0.4075  0.9119  1.5415  0.5677   NaN   
PPMI-92834  2.2357 -1.6479  ...  0.6015  0.0465  0.8480  1.2843  1.1177   NaN   

                                            
ASSAY         XCL1 XPNPEP2    YES1  ZBTB16  
PATNO                                       
PPMI-12224  0.9160 -0.7088  0.6516  0.6218  
PPMI-12499  1.0384  0.0735  4.1836  2.9935  
PPMI-12593  0.7270  1.8794  5.5804  4.7729  
PPMI-13039  1.8432 -1.9680  3.3025  2.9325  
PPMI-13424  0.9071  1.1510  1.9651  0.8021  
...            ...     ...     ...     ...  
PPMI-90456  1.0872  0.2158  2.8002  2.3953  
PPMI-91097  1.4145 -0.0319  4.8730  4.8874  
PPMI-91837  0.6329 -1.0747  5.0794  3.9947  
PPMI-92490  0.6276 -1.2518  3.2023  2.8394  
PPMI-92834  2.0662 -1.2608  5.9305  5.4363  

[190 rows x 370 columns]

In [487]:
olink_immune_ppmi2

NPX                                                          \
ASSAY        ACTN4     ADA  ADAM23  ADGRE2    AGER    AGRN    AGRP ALDH3A1   
PATNO                                                                        
PPMI-12224  0.5581  0.0255  0.4261  0.4567  0.4495  0.1113 -0.3477  1.0174   
PPMI-12499  0.4749  0.7887 -0.1354  0.5383 -0.5039  0.7472 -0.7027  0.6919   
PPMI-12593  0.4295  0.9949  1.1877  1.1618  1.1603  1.1894  0.3559  1.6790   
PPMI-13039  0.8661  0.8572  0.8249  0.6617  1.1946  0.9019  0.3256  0.9498   
PPMI-13424  0.5792  0.1836  1.3378  0.6832  1.1123  0.5034 -0.9368  0.9231   
...            ...     ...     ...     ...     ...     ...     ...     ...   
PPMI-90456  0.5225  0.3202  0.7993  1.1447  0.5764  1.1735  0.7613  0.4444   
PPMI-91097  0.3730  0.4138  0.7145  0.4270 -0.1674  0.5170 -0.4557  0.9253   
PPMI-91837  0.5791  1.0554  1.1387  0.9527  0.1538  0.6254  0.5831  0.9235   
PPMI-92490  0.6866  0.5069  0.7676  0.7375 -0.8044  0.2955 -0.8342  1.2890   
PPMI-92834  0.4074  0.1091  0.6417  0.0113  0.2610  0.2043 -0.4641  0.2759   

                            ...                                          \
ASSAY         AMBN     AMN  ...   TREM2  TRIM21   TRIM5   VASH1   VEGFA   
PATNO                       ...                                           
PPMI-12224  0.0776  0.7329  ...  1.0766  0.2489 -0.1871  0.0489  0.3028   
PPMI-12499 -0.0823  1.1827  ...  1.1865  2.6209  3.5037  2.3199  0.4727   
PPMI-12593  0.0008  1.0241  ...  1.7242  4.2532  3.1597  2.6634  2.3096   
PPMI-13039 -0.0352  1.2295  ...  1.4474  2.4389  2.3450  1.4662  0.7809   
PPMI-13424  0.2393  0.4682  ...  1.4521 -0.3102  1.1282  0.7613  0.6114   
...            ...     ...  ...     ...     ...     ...     ...     ...   
PPMI-90456  0.1835  1.2310  ...  2.3998  1.8158  1.9024  0.9528  0.9534   
PPMI-91097  0.6737  0.4551  ...  1.0371  2.6557  3.9497  3.4571  0.6584   
PPMI-91837  0.0753  1.4671  ...  0.9609  3.2496  4.4950  3.3186  1.3550   
PPMI-92490  0.2958  1.7638  ...  1.6177  1.6736  1.5642  1.1050  0.1619   
PPMI-92834  0.1670  1.6040  ...  0.2492  1.8418  3.1092  2.0270  1.2523   

                                                    
ASSAY        VEGFD     WAS WFIKKN2   WNT9A  YTHDF3  
PATNO                                               
PPMI-12224 -0.1967  0.2291  0.6022  1.0853 -1.2657  
PPMI-12499 -0.2292  4.3587  0.2105  1.7250  1.9232  
PPMI-12593  0.4485  5.3271  1.2661  1.7678  2.4497  
PPMI-13039  0.2878  3.1836  0.3638  2.1210  1.3587  
PPMI-13424  0.9236  1.6045  0.7239  1.7429 -0.1984  
...            ...     ...     ...     ...     ...  
PPMI-90456  0.2383  2.8996  0.7537  2.4754  0.7212  
PPMI-91097 -0.1563  5.1870 -0.4577  0.9576  1.9264  
PPMI-91837  0.5155  4.8107  0.0503  2.3010  2.9685  
PPMI-92490  0.2565  2.3329 -0.1432  1.5637  0.8520  
PPMI-92834  0.1424  3.9642  0.1300  1.3188  1.7506  

[215 rows x 368 columns]

In [488]:
olink_neuro_ppmi2

NPX                                                          \
ASSAY      ABHD14B  ACVRL1  ADAM22   ADAM8  ADGRB3     AFP    AGR2    AHSP   
PATNO                                                                        
PPMI-12224 -1.0427  0.3356  0.6593  1.1731  0.6793  0.4173  1.4495 -0.1450   
PPMI-12499  1.7170  0.2523 -0.0698  1.0587  0.1167  0.8347  1.0012  1.2912   
PPMI-12593  2.1564  0.8950  1.2445  1.7522  1.3080  0.5556  3.1745  1.3039   
PPMI-13039  0.4433  0.6081  0.8687  1.0532  1.0985  0.4090  1.5385 -0.1189   
PPMI-13424 -0.8059  0.5702  0.7677  0.9854  1.0053  0.2784  0.4172 -1.0076   
...            ...     ...     ...     ...     ...     ...     ...     ...   
PPMI-90456  0.1243  0.7322  0.8727  1.0671  0.9838 -0.7525  2.7987 -0.0204   
PPMI-91097  1.3765  0.5005  0.4387  0.7795  0.7202  0.8583  1.2787  0.0421   
PPMI-91837  1.3909  0.6812  0.3056  0.9380  0.4007  0.6162  2.1059  0.1769   
PPMI-92490  1.6601  0.4968  0.3732  0.9367  0.7301  1.0051  3.6140  2.5668   
PPMI-92834  1.7996  1.1910  0.4289  1.7294  0.7324 -0.5238  0.5893  2.8593   

                            ...                                          \
ASSAY       AKT1S1 ALDH1A1  ...   VSIG4   VSTM1    VTA1    VWC2    WARS   
PATNO                       ...                                           
PPMI-12224  0.4234  0.3734  ...  0.5687 -0.6831 -0.3290  0.6803  0.9381   
PPMI-12499  3.4608  1.5391  ...  0.0282  0.5855  3.2124  0.2157  1.9674   
PPMI-12593  4.0912  1.7893  ...  1.1570  1.0844  5.0271  1.2505  2.3615   
PPMI-13039  2.8359  1.4664  ...  0.7470  0.2701  2.4670  0.4463  1.8534   
PPMI-13424  1.5384  0.3253  ...  1.0424 -1.0625  1.5488  1.0303  1.3155   
...            ...     ...  ...     ...     ...     ...     ...     ...   
PPMI-90456  2.2364  0.2099  ...  1.6525 -0.1303  1.8542  0.9407  1.1742   
PPMI-91097  2.1678  1.2934  ...  1.0139 -0.0147  4.0234  0.6234  1.8670   
PPMI-91837  4.4684  0.9109  ...  1.2175 -0.4605  4.4542  0.4892  1.9226   
PPMI-92490  3.0499  1.8012  ...  0.3709  0.2709  2.8668  0.9354  1.1390   
PPMI-92834  3.9967  0.9762  ...  1.2217  1.5022  4.3810  0.1260  1.5087   

                                                  
ASSAY        WASF3 WFIKKN1 WISP2    WWP2   XRCC4  
PATNO                                             
PPMI-12224 -0.1111 -0.7623   NaN -0.1156 -0.0519  
PPMI-12499  2.8901 -0.9416   NaN  1.9975  0.1859  
PPMI-12593  4.9490 -0.6098   NaN  2.2069  0.6118  
PPMI-13039  3.4660  0.2255   NaN  1.5444  0.7564  
PPMI-13424 -0.4501 -0.9200   NaN  0.5509  0.2763  
...            ...     ...   ...     ...     ...  
PPMI-90456  1.8817 -0.6284   NaN  0.6445  0.3755  
PPMI-91097  3.2949 -0.5291   NaN  1.9553  0.7915  
PPMI-91837  4.0621 -0.9929   NaN  2.4613  0.8884  
PPMI-92490  1.0268 -0.1541   NaN  1.0256  0.5289  
PPMI-92834  3.5394 -0.5368   NaN  2.4823  1.1166  

[227 rows x 371 columns]

In [489]:
olink_neuro_ppmi2.columns = olink_neuro_ppmi2.columns.droplevel()

In [490]:
olink_cancer_ppmi2.columns = olink_cancer_ppmi2.columns.droplevel()
olink_immune_ppmi2.columns = olink_immune_ppmi2.columns.droplevel()
olink_cardio_ppmi2.columns = olink_cardio_ppmi2.columns.droplevel()

In [491]:
olink_cancer_ppmi2.shape

(190, 370)

In [493]:
#merge data 
merge = pd.merge(olink_cancer_ppmi2, olink_neuro_ppmi2, on="PATNO") 
merge = pd.merge(merge, olink_immune_ppmi2, on="PATNO")
#merge = pd.merge(merge, olink_cardio_ppmi2, on="PATNO")


In [494]:
merge = pd.merge(merge, olink_cardio_ppmi2, on="PATNO")

/tmp/ipykernel_793386/3976361491.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TNF_x', 'IL6_x', 'CXCL8_x'} in the result is deprecated and will raise a MergeError in a future version.
  merge = pd.merge(merge, olink_cardio_ppmi2, on="PATNO")


In [495]:
merge.columns.unique()
#DUPLICATES?

Index(['AARSD1', 'ABL1', 'ACAA1', 'ACP6', 'ADAMTS15', 'ADAMTS8', 'ADCYAP1R1',
       'ADGRG1', 'ADM', 'AGR3',
       ...
       'VAMP5', 'VASN', 'VCAM1', 'VIM', 'VSIR', 'VSTM2L', 'VWF', 'WASF1', 'XG',
       'ZBTB17'],
      dtype='object', name='ASSAY', length=1475)

In [496]:
merge

ASSAY,AARSD1,ABL1,ACAA1,ACP6,ADAMTS15,ADAMTS8,ADCYAP1R1,ADGRG1,ADM,AGR3,...,VAMP5,VASN,VCAM1,VIM,VSIR,VSTM2L,VWF,WASF1,XG,ZBTB17
PATNO,,,,,,,,,,,,,,,,,,,,,
PPMI-12224,0.3978,0.3545,-1.3954,1.0728,3.3188,1.0104,-0.3453,1.0168,1.5285,0.9903,...,0.8579,0.3840,0.1798,-1.2563,0.2103,0.3249,-0.2497,0.8843,0.6841,0.3738
PPMI-12499,2.9192,2.1646,1.2238,1.2234,2.9050,0.3143,0.4974,4.1220,1.5271,-3.8330,...,1.3394,0.4856,0.7560,0.5568,1.3699,-0.3368,-0.1646,3.1443,0.6019,0.7921
PPMI-12593,3.7517,3.8234,2.5279,1.1390,3.8457,1.7085,-0.1460,0.8952,1.7712,-0.1343,...,1.6105,0.9814,1.1077,0.6210,4.0803,0.3843,0.5903,4.1704,1.6873,1.4572
PPMI-13039,1.9736,1.7247,0.5997,0.8770,3.3143,0.8214,0.4348,2.1407,1.6113,0.3798,...,3.8250,0.5734,1.1865,0.4003,1.5482,-0.3134,-0.3715,2.6155,0.3041,1.4129
PPMI-13424,0.7901,0.8931,-0.2197,0.9044,3.4085,1.2802,-2.0693,-0.3079,1.4587,-2.0356,...,0.7310,0.8911,0.8082,1.0581,0.4278,0.1791,-0.0482,1.3633,1.3139,0.7831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PPMI-90456,2.0204,2.6473,0.3890,1.3523,3.1578,1.3138,-0.0015,1.8442,1.8755,0.7587,...,0.9492,0.6297,1.5695,0.0678,0.8510,-0.1613,0.2416,2.5308,0.7476,1.0249
PPMI-91097,2.9806,1.9931,1.9430,0.8259,3.0793,-0.1127,-1.6191,1.4559,1.4954,-0.5183,...,1.4275,0.5357,0.4415,0.0614,1.4755,0.5233,-0.7231,3.9334,0.5140,0.6333
PPMI-91837,3.6136,3.4301,2.3686,0.5248,3.8056,0.2230,-0.2413,1.8540,1.7689,-0.1283,...,1.7608,0.4475,0.5548,1.1027,1.6808,0.0236,0.2021,4.8072,0.7085,0.9884


In [161]:
#Some dup columns marked as _x remove on merger

InvalidIndexError: (slice(1, 10, None), slice(1, 10, None), slice(None, None, None))

In [180]:
merge

ASSAY,AARSD1,ABL1,ACAA1,ACP6,ADAMTS15,ADAMTS8,ADCYAP1R1,ADGRG1,ADM,AGR3,...,VAMP5,VASN,VCAM1,VIM,VSIR,VSTM2L,VWF,WASF1,XG,ZBTB17
PATNO,,,,,,,,,,,,,,,,,,,,,
CONTROL_SAMPLE_CSAS2-5,0.032700,0.558100,-0.432900,0.550200,0.581400,0.354600,0.250600,1.291800,0.460800,0.119600,...,-0.182400,0.333700,0.291300,0.327500,0.648600,0.089800,1.013100,0.236300,0.194200,-0.072300
CONTROL_SAMPLE_CSAS2-6,0.012500,0.135000,0.047600,0.660700,0.064000,0.170500,0.001000,1.014200,0.213200,0.403300,...,0.221400,-0.159900,0.112900,1.126300,0.031800,0.137900,0.792000,0.714700,0.421100,0.201600
PPMI-10874,1.831533,0.930400,0.086400,1.169733,2.917433,0.783067,-0.461367,1.108167,1.865733,-0.628533,...,2.454333,0.295567,0.721133,0.418200,0.934333,0.444967,0.317300,0.893267,1.411100,0.273733
PPMI-12224,0.770433,0.486367,-1.066267,1.085467,3.045267,1.145833,-0.197767,0.896667,1.482900,0.438000,...,0.813333,0.481000,0.384900,-0.765767,0.239033,0.211267,-0.387833,0.952900,0.888133,0.713400
PPMI-12499,2.948267,2.057467,1.587200,1.080433,2.764533,0.176267,-0.257867,3.947933,1.486700,-2.645333,...,1.707500,0.368400,0.687633,0.802900,1.981867,-0.158200,-0.231033,3.442767,0.499733,0.847700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PPMI-91837,3.613600,3.430100,2.368600,0.524800,3.805600,0.223000,-0.241300,1.854000,1.768900,-0.128300,...,1.760800,0.447500,0.554800,1.102700,1.680800,0.023600,0.202100,4.807200,0.708500,0.988400
PPMI-92490,3.395027,1.949118,1.052900,0.814100,3.011264,0.832091,-0.404009,3.480409,1.772045,0.622618,...,1.996018,0.316773,0.845891,0.412618,1.452273,0.152845,0.322209,2.957545,0.728300,1.077845
PPMI-92834,3.392767,1.708400,-0.198567,0.667300,2.589533,0.250767,-0.268367,1.811800,2.188867,-1.815267,...,0.541233,-0.613600,0.699600,0.286700,0.475067,0.199667,-1.370133,1.810167,-0.404167,1.604467


In [497]:
merge['id'] = merge.index.str.split('-').str[1]

In [504]:
merge.tail(10)

ASSAY,AARSD1,ABL1,ACAA1,ACP6,ADAMTS15,ADAMTS8,ADCYAP1R1,ADGRG1,ADM,AGR3,...,VASN,VCAM1,VIM,VSIR,VSTM2L,VWF,WASF1,XG,ZBTB17,id
PATNO,,,,,,,,,,,,,,,,,,,,,
PPMI-74977,4.6972,1.7995,0.8152,1.5975,3.2216,0.8929,-0.2018,-0.1899,1.3026,-1.4895,...,0.8893,0.6769,0.3022,1.4606,0.2295,0.0681,2.3511,1.4894,1.2742,74977
PPMI-75564,4.4308,2.5328,1.4192,1.3538,2.7058,1.8466,0.3800,2.4231,1.6432,-0.8686,...,0.4258,1.1530,0.0284,2.5072,3.9364,0.4151,3.5713,0.3036,1.3655,75564
PPMI-85062,3.8400,3.1773,1.6377,1.4813,3.7259,-0.1760,-0.1156,4.5467,2.1511,0.3071,...,0.5581,1.4902,2.3559,3.6098,0.6731,1.9415,4.0776,-0.0053,2.9845,85062
PPMI-85236,2.7653,1.5648,0.3786,1.3946,3.4556,0.1635,-0.8422,0.3895,1.3714,1.8154,...,0.9660,0.9533,0.6446,0.3013,0.5551,-0.0095,2.5917,0.5718,0.6838,85236
PPMI-85242,2.7856,2.1913,1.4809,0.4823,3.2580,0.3579,-0.6192,1.7481,1.4149,0.4920,...,0.0072,0.4960,0.4910,1.8551,0.6720,0.1511,3.6517,0.0416,0.9119,85242
PPMI-90456,2.0204,2.6473,0.3890,1.3523,3.1578,1.3138,-0.0015,1.8442,1.8755,0.7587,...,0.6297,1.5695,0.0678,0.8510,-0.1613,0.2416,2.5308,0.7476,1.0249,90456
PPMI-91097,2.9806,1.9931,1.9430,0.8259,3.0793,-0.1127,-1.6191,1.4559,1.4954,-0.5183,...,0.5357,0.4415,0.0614,1.4755,0.5233,-0.7231,3.9334,0.5140,0.6333,91097
PPMI-91837,3.6136,3.4301,2.3686,0.5248,3.8056,0.2230,-0.2413,1.8540,1.7689,-0.1283,...,0.4475,0.5548,1.1027,1.6808,0.0236,0.2021,4.8072,0.7085,0.9884,91837
PPMI-92490,3.2302,1.7680,0.5892,0.7021,3.2030,0.8842,0.1209,3.6206,1.9094,0.2387,...,0.1973,0.6053,-0.2981,0.7515,-0.2682,0.1151,1.9996,0.4364,0.7606,92490


In [196]:
#merge.drop(merge.tail(2).index,
 #       inplace = True)

In [505]:
merge['id'] = merge['id'].astype(np.float64)

In [506]:
merge["PATNO"] = merge['id']
merge.index.names = ['id1']

In [507]:
X_val["PATNO"]

0      3000.0
1      3001.0
2      3002.0
3      3003.0
4      3004.0
        ...  
424    4124.0
425    4125.0
426    4126.0
427    4136.0
428    4139.0
Name: PATNO, Length: 429, dtype: float64

In [508]:
X_val

,Age,Inferred_Gender,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,X1.155044197_T_G_G,...,X9.33778399_G_A_G,X9.34046391_C_T_C,X9.5023192_A_G_G,X9.5065003_C_G_G,X9.86258685_A_C_C,X9.86945961_A_C_C,X9.89056018_A_G_G,X9.89059858_G_A_A,X9.93256092_T_C_T,PATNO
0,0.735751,0.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,2,2,1,3000.0
1,0.659409,1.0,1,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3001.0
2,0.705926,0.0,2,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,3002.0
3,0.498510,0.0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3003.0
4,0.549998,1.0,1,1,0,0,1,0,1,0,...,1,0,0,0,1,0,1,1,0,3004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,0.773529,1.0,1,0,0,1,2,0,0,0,...,1,0,0,0,0,0,2,2,1,4124.0
425,0.637956,1.0,1,0,0,1,0,1,1,0,...,0,0,0,0,1,0,0,1,0,4125.0
426,0.479726,1.0,1,0,0,0,1,1,1,0,...,1,0,0,0,0,0,1,1,1,4126.0
427,0.480301,1.0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,1,1,2,4136.0


In [509]:
X_val1 = pd.merge(X_val, merge, on="PATNO") 

In [510]:
X_val1

,Age,Inferred_Gender,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,X1.155044197_T_G_G,...,VASN,VCAM1,VIM,VSIR,VSTM2L,VWF,WASF1,XG,ZBTB17,id
0,0.549998,1.0,1,1,0,0,1,0,1,0,...,0.70095,0.33595,1.03885,0.4021,-0.07850,0.1753,0.43780,0.31340,0.6594,3004.0
1,0.512735,1.0,2,0,0,0,1,1,1,0,...,0.48890,0.80770,0.14350,0.6405,-0.20190,0.3673,0.85210,0.62860,0.7099,3016.0
2,0.861435,1.0,0,0,0,1,1,1,1,0,...,0.30890,0.75910,-0.40600,-0.5907,0.14020,-0.6044,0.03080,2.95780,0.4402,3028.0
3,0.569724,0.0,1,0,0,0,0,1,1,0,...,-0.07085,0.15375,-1.57845,1.4830,0.56235,-1.6926,1.38625,1.23175,0.5938,3064.0
4,0.316501,0.0,1,0,0,2,1,0,0,0,...,0.65590,0.66770,-0.42400,1.2444,-0.00530,-1.2320,2.18430,1.56970,0.7003,3072.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.886000,0.0,0,0,0,1,1,0,0,0,...,0.46340,0.70570,0.45400,0.9610,-0.11910,0.9952,2.58290,1.11330,0.5641,4092.0
99,0.527971,1.0,1,0,0,0,1,1,1,0,...,0.45500,0.53120,0.17860,-0.0823,0.07440,-0.7243,1.57720,0.33280,0.8141,4100.0
100,0.686775,1.0,0,0,0,1,1,0,0,1,...,0.36830,1.07520,-0.16940,0.8876,0.82280,-0.3057,1.88950,-0.01790,0.9852,4115.0
101,0.637956,1.0,1,0,0,1,0,1,1,0,...,0.18690,0.76270,-0.12000,1.0455,0.01230,-0.4182,1.62790,0.26760,0.5158,4125.0


In [511]:
X_val1[["id"]].to_csv("X_val_PATNO.csv")

In [517]:
#Y_val1

,Y,PATNO
0,0.0,3000.0
1,1.0,3001.0
2,1.0,3002.0
3,1.0,3003.0
4,0.0,3004.0
...,...,...
425,1.0,4125.0
426,1.0,4126.0
427,1.0,4136.0
428,0.0,4139.0


In [513]:
Y_val["PATNO"] = X_val["PATNO"]

In [521]:
X_val["PATNO"]

0      3000.0
1      3001.0
2      3002.0
3      3003.0
4      3004.0
        ...  
424    4124.0
425    4125.0
426    4126.0
427    4136.0
428    4139.0
Name: PATNO, Length: 429, dtype: float64

In [531]:
Y_val

0                                                      0.0
1                                                      1.0
2                                                      1.0
3                                                      1.0
4                                                      0.0
                               ...                        
425                                                    1.0
426                                                    1.0
427                                                    1.0
428                                                    0.0
PATNO    0      3000.0
1      3001.0
2      3002.0
3   ...
Name: Y, Length: 430, dtype: object

In [533]:
Y_val
#STOPPED HERE DOUBLE CHECK

0      0.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
424    1.0
425    1.0
426    1.0
427    1.0
428    0.0
Name: Y, Length: 429, dtype: float64

In [534]:
Y_val1 = pd.DataFrame(Y_val.T, columns=['Y'])

In [535]:
Y_val1

,Y
0,0.0
1,1.0
2,1.0
3,1.0
4,0.0
...,...
424,1.0
425,1.0
426,1.0
427,1.0


In [536]:
Y_val1["PATNO"] = X_val["PATNO"]

In [537]:
Y_val1

,Y,PATNO
0,0.0,3000.0
1,1.0,3001.0
2,1.0,3002.0
3,1.0,3003.0
4,0.0,3004.0
...,...,...
424,1.0,4124.0
425,1.0,4125.0
426,1.0,4126.0
427,1.0,4136.0


In [538]:
Y_val1 = pd.merge(Y_val1, X_val1["PATNO"],  on="PATNO")

In [593]:
Y_val1


0      0.0
1      0.0
2      1.0
3      0.0
4      0.0
      ... 
98     1.0
99     0.0
100    1.0
101    1.0
102    1.0
Name: Y, Length: 103, dtype: float64

In [540]:
Y_val1 = Y_val1["Y"]

In [542]:
X_val1

,Age,Inferred_Gender,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,X1.155044197_T_G_G,...,VASN,VCAM1,VIM,VSIR,VSTM2L,VWF,WASF1,XG,ZBTB17,id
0,0.549998,1.0,1,1,0,0,1,0,1,0,...,0.70095,0.33595,1.03885,0.4021,-0.07850,0.1753,0.43780,0.31340,0.6594,3004.0
1,0.512735,1.0,2,0,0,0,1,1,1,0,...,0.48890,0.80770,0.14350,0.6405,-0.20190,0.3673,0.85210,0.62860,0.7099,3016.0
2,0.861435,1.0,0,0,0,1,1,1,1,0,...,0.30890,0.75910,-0.40600,-0.5907,0.14020,-0.6044,0.03080,2.95780,0.4402,3028.0
3,0.569724,0.0,1,0,0,0,0,1,1,0,...,-0.07085,0.15375,-1.57845,1.4830,0.56235,-1.6926,1.38625,1.23175,0.5938,3064.0
4,0.316501,0.0,1,0,0,2,1,0,0,0,...,0.65590,0.66770,-0.42400,1.2444,-0.00530,-1.2320,2.18430,1.56970,0.7003,3072.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.886000,0.0,0,0,0,1,1,0,0,0,...,0.46340,0.70570,0.45400,0.9610,-0.11910,0.9952,2.58290,1.11330,0.5641,4092.0
99,0.527971,1.0,1,0,0,0,1,1,1,0,...,0.45500,0.53120,0.17860,-0.0823,0.07440,-0.7243,1.57720,0.33280,0.8141,4100.0
100,0.686775,1.0,0,0,0,1,1,0,0,1,...,0.36830,1.07520,-0.16940,0.8876,0.82280,-0.3057,1.88950,-0.01790,0.9852,4115.0
101,0.637956,1.0,1,0,0,1,0,1,1,0,...,0.18690,0.76270,-0.12000,1.0455,0.01230,-0.4182,1.62790,0.26760,0.5158,4125.0


In [541]:
Y_val1.sum()

26.0

In [320]:
Y_val1

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
124    0.0
125    1.0
126    1.0
127    1.0
128    1.0
Name: Y, Length: 129, dtype: float64

In [ ]:
#Align olink data.

In [543]:
column_names = Trn_input.columns[637:]

In [254]:

#colnames(Trn_input)[637:] = column_names.str.split(';').str[0]

NameError: name 'colnames' is not defined

In [545]:
old_names = Trn_input.columns[637:]
Trn_input.rename(columns=dict(zip(old_names, old_names.str.split(';').str[0])), inplace=True)

In [544]:
old_names = Test_input.columns[637:]
Test_input.rename(columns=dict(zip(old_names, old_names.str.split(';').str[0])), inplace=True)

In [548]:
Trn_input

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,0,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3976,-0.1779,0.0035,-0.00010,1.53040,0.14805,0.1138,0.00205,-0.15445,-0.23840
1,0,63,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.5776,1.4763,1.6084,2.50490,0.20960,0.95065,1.7539,1.60820,2.95895,2.38200
2,0,54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.1263,0.4304,-0.5720,-0.11230,-0.79590,0.14650,0.0000,-1.20050,1.00125,0.40830
3,1,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0088,0.0434,-0.4618,-0.16240,0.57460,0.15085,-0.2558,-0.20285,-0.18495,-0.15640
4,1,41,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-0.4184,-0.7404,-0.4804,-0.39375,1.16155,0.29955,-2.4493,-1.99725,-0.15660,0.01445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26068,0,42,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.6161,1.4456,-0.8182,0.16810,-0.06440,0.57295,1.7542,0.23905,0.67695,0.45420
26069,0,53,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.3589,0.3378,-0.1453,0.22240,1.03260,0.07465,0.8922,0.16935,0.42315,0.18060
26070,1,65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2424,0.5050,-0.2181,0.01400,-1.21590,0.06865,1.2475,0.77360,0.56455,-0.40670
26071,1,54,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0208,0.4288,-0.0295,-0.56130,-2.15160,0.17025,0.7288,0.44765,0.04625,-0.12660


In [576]:
Test_input

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,1,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0949,-0.8319,-0.51690,-0.57360,0.26810,-0.74945,-0.6714,-1.66505,0.27345,0.18650
1,1,53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0302,-0.9410,0.16895,-0.32510,-0.08750,-0.22335,-1.7270,-0.86890,-0.86935,-0.58020
2,1,69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4265,0.2265,0.27950,-0.27475,0.07585,-0.37935,0.6809,0.46975,0.65270,0.17745
3,1,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0190,-0.3626,0.00635,0.01950,-0.59780,-0.11445,-0.6602,1.07940,-0.19845,0.15830
4,1,66,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.1431,-0.1234,0.36590,-0.59260,0.14490,-0.44585,-0.7857,0.11930,-0.19275,-0.19380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,1,64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1348,0.4933,-0.72360,0.10840,-1.26810,-0.00075,0.7743,2.24960,-0.01835,-0.25230
17331,0,69,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.4130,-0.2031,-0.60625,0.25610,-0.14920,-0.14885,-0.0388,0.35900,-0.50545,0.24840
17332,0,44,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0531,-0.3950,-0.61595,0.10330,0.36000,0.01115,-0.4016,-1.01480,0.01395,-0.31040
17333,0,67,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2045,0.1034,0.26015,0.45970,0.91660,-0.07995,0.2354,0.13150,0.26515,-0.29610


In [640]:
Test_input.iloc[1:2,0:638]

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,X21.39972727_C_A_A,X21.44357419_T_G_T,X22.22599537_G_A_A,X22.29656431_C_T_T,X22.39738425_A_C_A,X22.39738501_G_C_G,X22.39743170_G_T_G,X22.39758541_A_G_A,X22.39758881_A_G_A,AARSD1
1,1,53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0,0,1.0,1.0,1.0,0.0,0.0,-1.42025


In [ ]:
Test_input.iloc[1:2,0:638]

In [577]:
age_info = pd.concat([Trn_input, Test_input], axis=0)

In [583]:
#Min, Max, Mediian, Mean
age_info.Age.max()

70

In [586]:
age_info.Age.min()

40

In [585]:
age_info.Age.mean()

57.26322336896425

In [584]:
age_info.Age.median()

59.0

In [ ]:
#Min, Max, Mediian, Mean

In [549]:
names = Trn_input.columns

In [550]:
names

Index(['Inferred_Gender', 'Age', 'X1.8495945_T_C_C', 'X1.20149058_A_G_G',
       'X1.20149560_G_A_A', 'X1.20150839_A_G_G', 'X1.20150984_C_G_G',
       'X1.20150998_A_G_G', 'X1.20151290_C_A_A', 'X1.20151431_A_G_G',
       ...
       'WNT9A', 'WWP2', 'XCL1', 'XG', 'XPNPEP2', 'XRCC4', 'YES1', 'YTHDF3',
       'ZBTB16', 'ZBTB17'],
      dtype='object', length=2100)

In [551]:
X_val1[names]

KeyError: "['X1.20159823_T_G_G', 'X1.155070862_C_A_C', 'X1.155135036_G_A_A', 'X1.156007988_G_A_A', 'X1.161478859_T_C_T', 'X1.205723619_A_C_A', 'X2.136403994_C_T_T', 'X2.145801461_T_C_C', 'X2.148955732_C_T_T', 'X2.169110394_C_T_T', 'X2.170680879_G_A_A', 'X2.232796610_T_C_C', 'X3.49518023_G_A_G', 'X3.182897920_A_G_G', 'X4.893712_C_T_T', 'X4.77111032_C_T_T', 'X4.90606518_T_G_G', 'X4.90626111_G_A_G', 'X4.91340668_T_G_G', 'X5.60437381_A_G_G', 'X6.25759066_G_T_T', 'X6.27279852_T_C_C', 'X6.30116078_A_G_A', 'X6.30310500_T_C_C', 'X6.30314607_A_G_G', 'X6.30897774_A_G_G', 'X6.31106643_G_A_G', 'X6.31364707_G_A_A', 'X6.31365707_G_A_A', 'X6.31367865_G_A_A', 'X6.31369151_G_A_A', 'X6.31379931_G_A_A', 'X6.31380340_C_T_T', 'X6.31380449_C_T_T', 'X6.31388214_G_A_A', 'X6.31410948_C_T_T', 'X6.31435991_C_T_T', 'X6.31449778_A_G_G', 'X6.31622606_C_A_A', 'X6.32036055_C_T_T', 'X6.32062687_G_A_A', 'X6.32191339_C_T_T', 'X6.32212655_C_T_T', 'X6.32240875_A_G_A', 'X6.32273060_A_G_G', 'X6.32338986_C_T_T', 'X6.32343604

In [552]:
X_val1.columns.intersection(names)

Index(['Age', 'Inferred_Gender', 'X1.108176287_C_T_T', 'X1.151115007_G_A_A',
       'X1.154837939_A_G_G', 'X1.155026114_C_T_C', 'X1.155033317_T_C_T',
       'X1.155033918_C_T_T', 'X1.155034632_C_G_G', 'X1.155044197_T_G_G',
       ...
       'VAMP5', 'VASN', 'VCAM1', 'VIM', 'VSIR', 'VSTM2L', 'VWF', 'WASF1', 'XG',
       'ZBTB17'],
      dtype='object', length=2003)

In [553]:
X_val2 = X_val1[X_val1.columns.intersection(names)]

In [554]:
X_val2

,Age,Inferred_Gender,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,X1.155044197_T_G_G,...,VAMP5,VASN,VCAM1,VIM,VSIR,VSTM2L,VWF,WASF1,XG,ZBTB17
0,0.549998,1.0,1,1,0,0,1,0,1,0,...,0.40050,0.70095,0.33595,1.03885,0.4021,-0.07850,0.1753,0.43780,0.31340,0.6594
1,0.512735,1.0,2,0,0,0,1,1,1,0,...,0.66290,0.48890,0.80770,0.14350,0.6405,-0.20190,0.3673,0.85210,0.62860,0.7099
2,0.861435,1.0,0,0,0,1,1,1,1,0,...,1.38890,0.30890,0.75910,-0.40600,-0.5907,0.14020,-0.6044,0.03080,2.95780,0.4402
3,0.569724,0.0,1,0,0,0,0,1,1,0,...,-0.03435,-0.07085,0.15375,-1.57845,1.4830,0.56235,-1.6926,1.38625,1.23175,0.5938
4,0.316501,0.0,1,0,0,2,1,0,0,0,...,1.06060,0.65590,0.66770,-0.42400,1.2444,-0.00530,-1.2320,2.18430,1.56970,0.7003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.886000,0.0,0,0,0,1,1,0,0,0,...,1.08720,0.46340,0.70570,0.45400,0.9610,-0.11910,0.9952,2.58290,1.11330,0.5641
99,0.527971,1.0,1,0,0,0,1,1,1,0,...,0.98960,0.45500,0.53120,0.17860,-0.0823,0.07440,-0.7243,1.57720,0.33280,0.8141
100,0.686775,1.0,0,0,0,1,1,0,0,1,...,1.62470,0.36830,1.07520,-0.16940,0.8876,0.82280,-0.3057,1.88950,-0.01790,0.9852
101,0.637956,1.0,1,0,0,1,0,1,1,0,...,1.17180,0.18690,0.76270,-0.12000,1.0455,0.01230,-0.4182,1.62790,0.26760,0.5158


In [555]:
Trn_input1 = Trn_input[Trn_input.columns.intersection(X_val2.columns)]

In [556]:
Test_input1 = Test_input[Test_input.columns.intersection(X_val2.columns)]

In [557]:
Test_input1

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,1,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0949,-0.8319,-0.51690,-0.57360,0.26810,-0.74945,-0.6714,-1.66505,0.27345,0.18650
1,1,53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0302,-0.9410,0.16895,-0.32510,-0.08750,-0.22335,-1.7270,-0.86890,-0.86935,-0.58020
2,1,69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4265,0.2265,0.27950,-0.27475,0.07585,-0.37935,0.6809,0.46975,0.65270,0.17745
3,1,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0190,-0.3626,0.00635,0.01950,-0.59780,-0.11445,-0.6602,1.07940,-0.19845,0.15830
4,1,66,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.1431,-0.1234,0.36590,-0.59260,0.14490,-0.44585,-0.7857,0.11930,-0.19275,-0.19380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,1,64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1348,0.4933,-0.72360,0.10840,-1.26810,-0.00075,0.7743,2.24960,-0.01835,-0.25230
17331,0,69,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.4130,-0.2031,-0.60625,0.25610,-0.14920,-0.14885,-0.0388,0.35900,-0.50545,0.24840
17332,0,44,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0531,-0.3950,-0.61595,0.10330,0.36000,0.01115,-0.4016,-1.01480,0.01395,-0.31040
17333,0,67,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2045,0.1034,0.26015,0.45970,0.91660,-0.07995,0.2354,0.13150,0.26515,-0.29610


In [558]:
#standardize age and move inferred gender to front for ppmi
column = "Age"

In [559]:
#Standarize age, and biomarkers
Trn_input1[column] = (Trn_input1[column] - Trn_input1[column].min()) / (Trn_input1[column].max() - Trn_input1[column].min())
Test_input1[column] = (Test_input1[column] - Test_input1[column].min()) / (Test_input1[column].max() - Test_input1[column].min())
#X_val[column] = (X_val[column] - X_val[column].min()) / (X_val[column].max() - X_val[column].min())

/tmp/ipykernel_793386/2651736735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Trn_input1[column] = (Trn_input1[column] - Trn_input1[column].min()) / (Trn_input1[column].max() - Trn_input1[column].min())
/tmp/ipykernel_793386/2651736735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_input1[column] = (Test_input1[column] - Test_input1[column].min()) / (Test_input1[column].max() - Test_input1[column].min())


In [560]:
Test_input1

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,1,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0949,-0.8319,-0.51690,-0.57360,0.26810,-0.74945,-0.6714,-1.66505,0.27345,0.18650
1,1,0.433333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0302,-0.9410,0.16895,-0.32510,-0.08750,-0.22335,-1.7270,-0.86890,-0.86935,-0.58020
2,1,0.966667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4265,0.2265,0.27950,-0.27475,0.07585,-0.37935,0.6809,0.46975,0.65270,0.17745
3,1,0.633333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0190,-0.3626,0.00635,0.01950,-0.59780,-0.11445,-0.6602,1.07940,-0.19845,0.15830
4,1,0.866667,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.1431,-0.1234,0.36590,-0.59260,0.14490,-0.44585,-0.7857,0.11930,-0.19275,-0.19380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,1,0.800000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1348,0.4933,-0.72360,0.10840,-1.26810,-0.00075,0.7743,2.24960,-0.01835,-0.25230
17331,0,0.966667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.4130,-0.2031,-0.60625,0.25610,-0.14920,-0.14885,-0.0388,0.35900,-0.50545,0.24840
17332,0,0.133333,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0531,-0.3950,-0.61595,0.10330,0.36000,0.01115,-0.4016,-1.01480,0.01395,-0.31040
17333,0,0.900000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2045,0.1034,0.26015,0.45970,0.91660,-0.07995,0.2354,0.13150,0.26515,-0.29610


In [ ]:
Test_input.iloc[1:2,0:638]

In [646]:
sample10 = Test_input.iloc[:, 637:]

In [648]:
sample10.columns

Index(['AARSD1', 'ABHD14B', 'ABL1', 'ACAA1', 'ACAN', 'ACE2', 'ACOX1', 'ACP5',
       'ACP6', 'ACTA2',
       ...
       'WNT9A', 'WWP2', 'XCL1', 'XG', 'XPNPEP2', 'XRCC4', 'YES1', 'YTHDF3',
       'ZBTB16', 'ZBTB17'],
      dtype='object', length=1463)

In [561]:
#Save TRAIN, TEST, VAL

In [562]:
Trn_input1

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,0,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3976,-0.1779,0.0035,-0.00010,1.53040,0.14805,0.1138,0.00205,-0.15445,-0.23840
1,0,0.766667,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.5776,1.4763,1.6084,2.50490,0.20960,0.95065,1.7539,1.60820,2.95895,2.38200
2,0,0.466667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.1263,0.4304,-0.5720,-0.11230,-0.79590,0.14650,0.0000,-1.20050,1.00125,0.40830
3,1,0.633333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0088,0.0434,-0.4618,-0.16240,0.57460,0.15085,-0.2558,-0.20285,-0.18495,-0.15640
4,1,0.033333,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-0.4184,-0.7404,-0.4804,-0.39375,1.16155,0.29955,-2.4493,-1.99725,-0.15660,0.01445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26068,0,0.066667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.6161,1.4456,-0.8182,0.16810,-0.06440,0.57295,1.7542,0.23905,0.67695,0.45420
26069,0,0.433333,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.3589,0.3378,-0.1453,0.22240,1.03260,0.07465,0.8922,0.16935,0.42315,0.18060
26070,1,0.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2424,0.5050,-0.2181,0.01400,-1.21590,0.06865,1.2475,0.77360,0.56455,-0.40670
26071,1,0.466667,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0208,0.4288,-0.0295,-0.56130,-2.15160,0.17025,0.7288,0.44765,0.04625,-0.12660


In [563]:
X_val2

,Age,Inferred_Gender,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,X1.155044197_T_G_G,...,VAMP5,VASN,VCAM1,VIM,VSIR,VSTM2L,VWF,WASF1,XG,ZBTB17
0,0.549998,1.0,1,1,0,0,1,0,1,0,...,0.40050,0.70095,0.33595,1.03885,0.4021,-0.07850,0.1753,0.43780,0.31340,0.6594
1,0.512735,1.0,2,0,0,0,1,1,1,0,...,0.66290,0.48890,0.80770,0.14350,0.6405,-0.20190,0.3673,0.85210,0.62860,0.7099
2,0.861435,1.0,0,0,0,1,1,1,1,0,...,1.38890,0.30890,0.75910,-0.40600,-0.5907,0.14020,-0.6044,0.03080,2.95780,0.4402
3,0.569724,0.0,1,0,0,0,0,1,1,0,...,-0.03435,-0.07085,0.15375,-1.57845,1.4830,0.56235,-1.6926,1.38625,1.23175,0.5938
4,0.316501,0.0,1,0,0,2,1,0,0,0,...,1.06060,0.65590,0.66770,-0.42400,1.2444,-0.00530,-1.2320,2.18430,1.56970,0.7003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.886000,0.0,0,0,0,1,1,0,0,0,...,1.08720,0.46340,0.70570,0.45400,0.9610,-0.11910,0.9952,2.58290,1.11330,0.5641
99,0.527971,1.0,1,0,0,0,1,1,1,0,...,0.98960,0.45500,0.53120,0.17860,-0.0823,0.07440,-0.7243,1.57720,0.33280,0.8141
100,0.686775,1.0,0,0,0,1,1,0,0,1,...,1.62470,0.36830,1.07520,-0.16940,0.8876,0.82280,-0.3057,1.88950,-0.01790,0.9852
101,0.637956,1.0,1,0,0,1,0,1,1,0,...,1.17180,0.18690,0.76270,-0.12000,1.0455,0.01230,-0.4182,1.62790,0.26760,0.5158


In [564]:
first_column = X_val2.pop('Inferred_Gender') 
X_val2.insert(0,'Inferred_Gender', first_column)

In [565]:
X_val2

,Inferred_Gender,Age,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,X1.155044197_T_G_G,...,VAMP5,VASN,VCAM1,VIM,VSIR,VSTM2L,VWF,WASF1,XG,ZBTB17
0,1.0,0.549998,1,1,0,0,1,0,1,0,...,0.40050,0.70095,0.33595,1.03885,0.4021,-0.07850,0.1753,0.43780,0.31340,0.6594
1,1.0,0.512735,2,0,0,0,1,1,1,0,...,0.66290,0.48890,0.80770,0.14350,0.6405,-0.20190,0.3673,0.85210,0.62860,0.7099
2,1.0,0.861435,0,0,0,1,1,1,1,0,...,1.38890,0.30890,0.75910,-0.40600,-0.5907,0.14020,-0.6044,0.03080,2.95780,0.4402
3,0.0,0.569724,1,0,0,0,0,1,1,0,...,-0.03435,-0.07085,0.15375,-1.57845,1.4830,0.56235,-1.6926,1.38625,1.23175,0.5938
4,0.0,0.316501,1,0,0,2,1,0,0,0,...,1.06060,0.65590,0.66770,-0.42400,1.2444,-0.00530,-1.2320,2.18430,1.56970,0.7003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.0,0.886000,0,0,0,1,1,0,0,0,...,1.08720,0.46340,0.70570,0.45400,0.9610,-0.11910,0.9952,2.58290,1.11330,0.5641
99,1.0,0.527971,1,0,0,0,1,1,1,0,...,0.98960,0.45500,0.53120,0.17860,-0.0823,0.07440,-0.7243,1.57720,0.33280,0.8141
100,1.0,0.686775,0,0,0,1,1,0,0,1,...,1.62470,0.36830,1.07520,-0.16940,0.8876,0.82280,-0.3057,1.88950,-0.01790,0.9852
101,1.0,0.637956,1,0,0,1,0,1,1,0,...,1.17180,0.18690,0.76270,-0.12000,1.0455,0.01230,-0.4182,1.62790,0.26760,0.5158


In [303]:
#X_val2.fillna(X_val2.mode().iloc[0], inplace=True)

/tmp/ipykernel_793386/3911273629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val2.fillna(Trn_input.mode().iloc[0], inplace=True)


In [294]:
# #Save TRAIN, TEST, VAL

# Trn_input1.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_trainolink")
# Trn_target.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_trainolink")
# Test_input1.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_testolink")
# Test_target.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_testolink")

In [566]:
Test_input1

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,1,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0949,-0.8319,-0.51690,-0.57360,0.26810,-0.74945,-0.6714,-1.66505,0.27345,0.18650
1,1,0.433333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0302,-0.9410,0.16895,-0.32510,-0.08750,-0.22335,-1.7270,-0.86890,-0.86935,-0.58020
2,1,0.966667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4265,0.2265,0.27950,-0.27475,0.07585,-0.37935,0.6809,0.46975,0.65270,0.17745
3,1,0.633333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0190,-0.3626,0.00635,0.01950,-0.59780,-0.11445,-0.6602,1.07940,-0.19845,0.15830
4,1,0.866667,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.1431,-0.1234,0.36590,-0.59260,0.14490,-0.44585,-0.7857,0.11930,-0.19275,-0.19380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,1,0.800000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1348,0.4933,-0.72360,0.10840,-1.26810,-0.00075,0.7743,2.24960,-0.01835,-0.25230
17331,0,0.966667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.4130,-0.2031,-0.60625,0.25610,-0.14920,-0.14885,-0.0388,0.35900,-0.50545,0.24840
17332,0,0.133333,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0531,-0.3950,-0.61595,0.10330,0.36000,0.01115,-0.4016,-1.01480,0.01395,-0.31040
17333,0,0.900000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2045,0.1034,0.26015,0.45970,0.91660,-0.07995,0.2354,0.13150,0.26515,-0.29610


In [567]:
Trn_input1

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,0,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3976,-0.1779,0.0035,-0.00010,1.53040,0.14805,0.1138,0.00205,-0.15445,-0.23840
1,0,0.766667,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.5776,1.4763,1.6084,2.50490,0.20960,0.95065,1.7539,1.60820,2.95895,2.38200
2,0,0.466667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.1263,0.4304,-0.5720,-0.11230,-0.79590,0.14650,0.0000,-1.20050,1.00125,0.40830
3,1,0.633333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0088,0.0434,-0.4618,-0.16240,0.57460,0.15085,-0.2558,-0.20285,-0.18495,-0.15640
4,1,0.033333,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-0.4184,-0.7404,-0.4804,-0.39375,1.16155,0.29955,-2.4493,-1.99725,-0.15660,0.01445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26068,0,0.066667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.6161,1.4456,-0.8182,0.16810,-0.06440,0.57295,1.7542,0.23905,0.67695,0.45420
26069,0,0.433333,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.3589,0.3378,-0.1453,0.22240,1.03260,0.07465,0.8922,0.16935,0.42315,0.18060
26070,1,0.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2424,0.5050,-0.2181,0.01400,-1.21590,0.06865,1.2475,0.77360,0.56455,-0.40670
26071,1,0.466667,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0208,0.4288,-0.0295,-0.56130,-2.15160,0.17025,0.7288,0.44765,0.04625,-0.12660


In [296]:
#Test_target

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
17330    0.0
17331    0.0
17332    0.0
17333    0.0
17334    0.0
Name: Y, Length: 17335, dtype: float64

In [568]:
combined_ppmi_uk_target = pd.concat([Test_target, Y_val1])

In [569]:
combined_ppmi_uk_target_input = pd.concat([Test_input1, X_val2])

In [570]:
combined_ppmi_uk_target_input

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,1.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0949,-0.8319,-0.51690,-0.57360,0.26810,-0.74945,-0.6714,-1.66505,0.27345,0.18650
1,1.0,0.433333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0302,-0.9410,0.16895,-0.32510,-0.08750,-0.22335,-1.7270,-0.86890,-0.86935,-0.58020
2,1.0,0.966667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4265,0.2265,0.27950,-0.27475,0.07585,-0.37935,0.6809,0.46975,0.65270,0.17745
3,1.0,0.633333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0190,-0.3626,0.00635,0.01950,-0.59780,-0.11445,-0.6602,1.07940,-0.19845,0.15830
4,1.0,0.866667,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.1431,-0.1234,0.36590,-0.59260,0.14490,-0.44585,-0.7857,0.11930,-0.19275,-0.19380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.0,0.886000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.9024,1.2635,0.46850,1.11330,-0.22300,0.33150,4.1097,0.62710,1.57900,0.56410
99,1.0,0.527971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.9092,0.5487,0.34120,0.33280,1.17510,0.60260,2.5227,-0.07900,2.16260,0.81410
100,1.0,0.686775,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.7824,0.7934,0.92760,-0.01790,0.09070,0.54260,2.3997,0.25610,2.79610,0.98520
101,1.0,0.637956,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.3432,0.6003,1.57710,0.26760,1.37600,0.06980,2.4445,0.27170,1.35440,0.51580


In [305]:
# combined_ppmi_uk_target_input.to_pickle("/home/ffc4001/ukbbl)
# combined_ppmi_uk_target.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_combinedtest")

In [571]:
x_val2 = X_val2[Test_input1.columns]
x_val2.fillna(x_val2.mode().iloc[0], inplace=True)

In [572]:
#x_val2.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_updated_X_valolink")
#Y_val1.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_updated_Y_valolink")

In [318]:
x_val2

,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,X1.20151431_A_G_G,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,0.0,0.735751,1,1,1,1,1,1,1,1,...,1.851333,-0.158233,0.821533,0.774767,-1.134933,0.120867,0.621333,3.031033,1.383267,0.596400
1,1.0,0.549998,1,0,0,0,0,0,0,0,...,1.258867,0.578350,0.751100,0.359683,-1.683267,0.266400,1.447867,0.362567,1.914900,0.608200
2,0.0,0.979271,2,0,0,0,0,0,0,0,...,1.401533,1.280533,1.179933,0.566200,1.214600,0.453767,2.144767,-0.099133,1.874000,0.934900
3,0.0,0.929645,1,1,1,1,1,1,1,1,...,1.883967,-0.105967,0.649533,1.038300,0.748933,0.305400,0.643767,-1.387033,0.373433,1.141150
4,1.0,0.512735,0,1,1,1,1,1,1,1,...,1.870867,0.020233,1.421067,0.796200,1.397167,0.354533,1.196833,-0.648433,1.044900,1.426967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,1.0,0.527971,0,0,0,0,0,0,0,0,...,0.855100,0.825800,0.128267,0.189433,1.032400,0.689367,3.082200,0.859967,2.590833,0.848800
125,1.0,0.686775,1,0,0,0,0,0,0,0,...,1.590967,0.475700,0.939667,0.067967,-0.063633,0.269567,1.711067,0.040833,1.966833,1.100967
126,0.0,0.559887,1,2,2,2,2,2,2,2,...,1.347567,0.637300,0.744033,1.270333,0.478600,0.255267,2.766733,0.890167,1.876967,0.408767
127,1.0,0.637956,2,0,0,0,0,0,0,0,...,1.993467,0.690533,1.868833,-0.125433,1.377600,0.309133,2.269900,1.415533,1.465867,0.705700


In [573]:
temp3 = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_combinedtest")

In [575]:
#temp3.columns.intersection(x_val2.columns)

Index(['Inferred_Gender', 'Age', 'X1.8495945_T_C_C', 'X1.20149058_A_G_G',
       'X1.20149560_G_A_A', 'X1.20150839_A_G_G', 'X1.20150984_C_G_G',
       'X1.20150998_A_G_G', 'X1.20151290_C_A_A', 'X1.20151431_A_G_G',
       ...
       'WNT9A', 'WWP2', 'XCL1', 'XG', 'XPNPEP2', 'XRCC4', 'YES1', 'YTHDF3',
       'ZBTB16', 'ZBTB17'],
      dtype='object', length=2003)

In [ ]:
#make ppmi updrs score training test


In [ ]:
#Seperate ppmi to test
#improve variant neural network
#Figures

In [319]:
!jupyter nbconvert --to html Preprocessing_olink.ipynb

[NbConvertApp] Converting notebook Preprocessing_olink.ipynb to html
[NbConvertApp] Writing 889843 bytes to Preprocessing_olink.html


In [590]:
Val_input

,Age,Inferred_Gender,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,X1.155044197_T_G_G,...,X9.33778399_G_A_G,X9.34046391_C_T_C,X9.5023192_A_G_G,X9.5065003_C_G_G,X9.86258685_A_C_C,X9.86945961_A_C_C,X9.89056018_A_G_G,X9.89059858_G_A_A,X9.93256092_T_C_T,eid_correct
0,69.139726,0.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,2,2,1,3000.0
1,65.142466,1.0,1,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3001.0
2,67.578082,0.0,2,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,3002.0
3,56.717808,0.0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3003.0
4,59.413699,1.0,1,1,0,0,1,0,1,0,...,1,0,0,0,1,0,1,1,0,3004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,71.117808,1.0,1,0,0,1,2,0,0,0,...,1,0,0,0,0,0,2,2,1,4124.0
425,64.019178,1.0,1,0,0,1,0,1,1,0,...,0,0,0,0,1,0,0,1,0,4125.0
426,55.734247,1.0,1,0,0,0,1,1,1,0,...,1,0,0,0,0,0,1,1,1,4126.0
427,55.764384,1.0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,1,1,2,4136.0


In [588]:
ppmi_id

,eid_correct
0,3000.0
1,3001.0
2,3002.0
3,3003.0
4,3004.0
...,...
424,4124.0
425,4125.0
426,4126.0
427,4136.0


In [591]:
demographics_ppmi = pd.concat([ppmi_id, Val_input], axis=1)

In [592]:
demographics_ppmi

,eid_correct,Age,Inferred_Gender,X1.108176287_C_T_T,X1.151115007_G_A_A,X1.154837939_A_G_G,X1.155026114_C_T_C,X1.155033317_T_C_T,X1.155033918_C_T_T,X1.155034632_C_G_G,...,X9.33778399_G_A_G,X9.34046391_C_T_C,X9.5023192_A_G_G,X9.5065003_C_G_G,X9.86258685_A_C_C,X9.86945961_A_C_C,X9.89056018_A_G_G,X9.89059858_G_A_A,X9.93256092_T_C_T,eid_correct
0,3000.0,69.139726,0.0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,2,2,1,3000.0
1,3001.0,65.142466,1.0,1,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,3001.0
2,3002.0,67.578082,0.0,2,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,3002.0
3,3003.0,56.717808,0.0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,3003.0
4,3004.0,59.413699,1.0,1,1,0,0,1,0,1,...,1,0,0,0,1,0,1,1,0,3004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,4124.0,71.117808,1.0,1,0,0,1,2,0,0,...,1,0,0,0,0,0,2,2,1,4124.0
425,4125.0,64.019178,1.0,1,0,0,1,0,1,1,...,0,0,0,0,1,0,0,1,0,4125.0
426,4126.0,55.734247,1.0,1,0,0,0,1,1,1,...,1,0,0,0,0,0,1,1,1,4126.0
427,4136.0,55.764384,1.0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,1,1,2,4136.0


In [616]:
demographics_ppmi["eid_correct1"] =demographics_ppmi["eid_correct"].iloc[:, 0]

In [617]:
fdemographics_ppmi["eid_correct1"]

0      3000.0
1      3001.0
2      3002.0
3      3003.0
4      3004.0
        ...  
424    4124.0
425    4125.0
426    4126.0
427    4136.0
428    4139.0
Name: eid_correct1, Length: 429, dtype: float64

In [596]:
X_val1["PATNO"]

0      3004.0
1      3016.0
2      3028.0
3      3064.0
4      3072.0
        ...  
98     4092.0
99     4100.0
100    4115.0
101    4125.0
102    4126.0
Name: PATNO, Length: 103, dtype: float64

In [619]:
demo_ppmi = demographics_ppmi[demographics_ppmi.eid_correct1.isin(X_val1.PATNO)]

In [599]:
X_val1.PATNO

0      3004.0
1      3016.0
2      3028.0
3      3064.0
4      3072.0
        ...  
98     4092.0
99     4100.0
100    4115.0
101    4125.0
102    4126.0
Name: PATNO, Length: 103, dtype: float64

In [621]:
demo_ppmi.Age.max()

82.712328767

In [625]:
demo_ppmi.Age.min()

30.615824538

In [626]:
demo_ppmi.Age.mean()

60.41142628107766

In [627]:
demo_ppmi.Age.median()

60.454083389

In [628]:
demo_ppmi.Inferred_Gender.value_counts()

1.0    74
0.0    29
Name: Inferred_Gender, dtype: int64